In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm_notebook as tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import (AutoTokenizer, AutoModel, AdamW, BertTokenizerFast,
                          T5ForConditionalGeneration, T5Tokenizer)
from sklearn.model_selection import train_test_split

## BERT

In [2]:
class QuestionsClassifier(nn.Module):
    def __init__(self, n_classes):
        super(QuestionsClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(name_model)
        self.drop = nn.Dropout(p=0.2)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        output = self.drop(outputs['last_hidden_state'][:,0,:])
        return self.out(output)

Load our fine tuned BERT model

In [3]:
name_model = "cointegrated/LaBSE-en-ru"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

n_class = 2
bert_model = QuestionsClassifier(n_class)
bert_tokenizer = BertTokenizerFast.from_pretrained(name_model)


Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
bert_model.load_state_dict(torch.load('./classification_bert1.bin', map_location=device))
bert_model.eval();

In [5]:
def get_prediction(question, model=bert_model, tokenizer=bert_tokenizer, max_length=300):
    """
    Predict a class for one question
    """
    model = model.eval()
    
    with torch.no_grad():
        input = tokenizer.encode_plus(
                        question,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=max_length,
                        return_token_type_ids=False,
                        padding='max_length'
                    )
        ids = input['input_ids']
        mask = input['attention_mask']

        input_ids = torch.tensor(ids, dtype=torch.long).reshape(1,-1)
        attention_mask = torch.tensor(mask, dtype=torch.long).reshape(1,-1)

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask
        )

        pred = torch.argmax(outputs)
        
    return pred.cpu().numpy()

## T51

Load our fine tuned T51 model

In [6]:
path = 'rut51'
model51 = T5ForConditionalGeneration.from_pretrained(path)
tokenizer51 = T5Tokenizer.from_pretrained(path)


In [7]:
model51.eval()
def answer51(x, **kwargs):
    """ 
    A function that generates an answer to a question
    """
    inputs = tokenizer51(x, return_tensors='pt').to(model51.device)
    
    with torch.no_grad():
        hypotheses = model51.generate(**inputs, **kwargs)
    return tokenizer51.decode(hypotheses[0], skip_special_tokens=True)

## Generating responses

In [8]:
def qa_system(question):
    """
    Checks the class of the question and generates an answer
    """
    class_pred = get_prediction(question)
    
    
    if class_pred == 1:
        return answer51(question)
    else:
        return "Вопрос был передан оператору."

In [9]:
qa_system('Добрый день, подскажите пожалуйста, у меня ведь будет возможность подготовиться к экзамену перед очной сдачей: открыть заранее все программы и пособие с курса?')

'Да, будет возможность.'

In [10]:
qa_system(' Здравствуйте, прошу добавить попытки на 2 задание в десятом упражнении. Спасибо!')

'Попытки добавлены.'

In [11]:
qa_system('Здравствуйте! Можно ли использовать Google Colab для решения заданий на прокторинге?')

'Возможно.'

In [12]:
qa_system('Добрый день! Мы бы хотели пройти бесплатное онлайн IT обучение.')

'Вопрос был передан оператору.'

In [13]:
qa_system('Здравствуйте, хотел спросить ? Для получения сертификата мне надо чтобы все упражнения были выполнены больше чем на 60% или чтоб Тотал всех выполненных заданий был больше чем 60% ???')

'Вопрос был передан оператору.'